In [1]:
import json 
import numpy as np 
import pandas as pd 
import os 
from glob import glob 
from IPython.display import clear_output

In [2]:
tanishq_files = glob(r'../data/raw/*anishq*.json') # This is problematic 
tanishq_files = ['../data/raw/ekatvambytanishq.json',
                 '../data/raw/BoycottTanishqJewelry.json',
                 '../data/raw/boycott_tanishq.json',
                 '../data/raw/BoycottTanishq2.json',
                 '../data/raw/BoycottTanishq.json']

tweets = []
for file in tanishq_files:
    print(f'Processing file: {file}')
    with open(file, encoding='utf-8') as f:
        temp = json.load(f)
        tweets.extend([tweet['full_text'] for tweet in temp if tweet['full_text'].isascii()])

print(f'Number of tweets extracted: {len(tweets)}')
n_tweets = len(tweets)
tanishq_data = pd.DataFrame(tweets, columns=['Tweet'])

del tweets

Processing file: ../data/raw/ekatvambytanishq.json
Processing file: ../data/raw/BoycottTanishqJewelry.json
Processing file: ../data/raw/boycott_tanishq.json
Processing file: ../data/raw/BoycottTanishq2.json
Processing file: ../data/raw/BoycottTanishq.json
Number of tweets extracted: 9427


In [3]:
if os.path.exists('labelled_dict_tanishq.npy'):
    labelled_dict_tanishq = np.load('labelled_dict_tanishq.npy', allow_pickle=True)[()]
else:
    labelled_dict_tanishq = tanishq_data["Tweet"].to_dict()
    labelled_dict_tanishq = {k: {"text": v, "sentiment": None} for k, v in labelled_dict_tanishq.items()}

In [4]:
count = 0
count_sent = 0
count_garbage = 0
for k, v in labelled_dict_tanishq.items():
    clear_output()
    count += 1
    if v['sentiment'] is not None:
        if v['sentiment'] == 10:
            count_garbage += 1
        else:
            count_sent += 1
        continue
    print('\n')
    print(f'{count} tweetes of {n_tweets} has been processed.')
    print(f'legal entries: {count_sent} garbage entries: {count_garbage}')
    print(f'Working on key: {k}')
    print(v['text'])
    v['sentiment'] = int(input("Please enter the sentiment:"))
    if v['sentiment'] == 10:
        count_garbage += 1
    else:
        count_sent += 1



2204 tweetes of 9427 has been processed.
legal entries: 372 garbage entries: 1831
Working on key: 2203
@ShashiTharoor @dhruv_rathee After nearly a month, #BoycottTanishq is trending yet again across social media platforms. This time, it is for an ad that roots for a ban on #firecrackers as part of its #Diwali campaign. 
#FirecrackersBan #Tanishq 
https://t.co/bEgJ9KFt6v


ValueError: invalid literal for int() with base 10: 'ee'

In [5]:
np.save('labelled_dict_tanishq.npy', labelled_dict_tanishq)

In [7]:
tanishq_data['sentiment'] = tanishq_data.index.to_series().map({k: v['sentiment'] for k, v in labelled_dict_tanishq.items()})

In [9]:
tanishq_data.to_csv('../data/processed/tanishq_data_labelled.csv')